# Testing the Deterministic Bank Statement Parser

This notebook demonstrates how to use the deterministic parser to extract transaction data from DBS bank statement PDFs.

## Features Demonstrated
- Basic parsing and extraction
- Confidence scoring and warnings
- Transaction data structure
- Validation against known values
- Visual inspection of extracted data

## Setup and Imports

In [1]:
import sys
import json
from pathlib import Path
import pandas as pd

# Add src to path so we can import our parser
sys.path.insert(0, str(Path('..').resolve()))

from src.parsers.deterministic_parser import DeterministicBankStatementParser

## 1. Basic Parsing Example

Let's parse the DBS Singapore bank statement and examine the results.

In [2]:
# Initialize the parser
parser = DeterministicBankStatementParser()

# Path to the PDF
pdf_path = "../resources/statements/DBS_POSB Consolidated Statement_Oct2025.pdf"

# Parse the document
result = parser.parse(pdf_path)

# Display basic results
print(f"Success: {result.success}")
print(f"Confidence: {result.confidence:.2%}")
print(f"Transactions Extracted: {len(result.data)}")
print(f"Warnings: {len(result.warnings)}")

if result.abort_reason:
    print(f"Abort Reason: {result.abort_reason}")

Success: True
Confidence: 100.00%
Transactions Extracted: 36
Warnings: 2


## 2. Examine Warnings

The parser generates warnings for anomalies or issues encountered during parsing.

In [3]:
print("Warnings:")
print("=" * 60)
for i, warning in enumerate(result.warnings, 1):
    print(f"{i}. {warning}")

Warnings:
1. Could not find transaction table on page 1
2. Could not find transaction table on page 6


## 3. View Sample Transactions

Let's examine the first few transactions to understand the data structure.

In [4]:
# Display first 5 transactions
print("First 5 Transactions:")
print("=" * 100)

for i, txn in enumerate(result.data[:5], 1):
    print(f"\n[Transaction {i}]")
    print(f"  Date:        {txn['date']}")
    print(f"  Description: {txn['description'][:60]}..." if len(txn['description']) > 60 else f"  Description: {txn['description']}")
    print(f"  Withdrawal:  {txn['withdrawal'] if txn['withdrawal'] else '-'}")
    print(f"  Deposit:     {txn['deposit'] if txn['deposit'] else '-'}")
    print(f"  Balance:     {txn['balance']}")
    print(f"  Page:        {txn['page']}")

First 5 Transactions:

[Transaction 1]
  Date:        01/10/2025
  Description: FAST Payment / Receipt 1,900.00 UOB:7803927190:I-BANK TRANSF...
  Withdrawal:  -
  Deposit:     -
  Balance:     337.7
  Page:        2

[Transaction 2]
  Date:        01/10/2025
  Description: FAST Payment / Receipt INCOMING PAYNOW REF 3052083 FROM: BER...
  Withdrawal:  -
  Deposit:     9.8
  Balance:     347.5
  Page:        2

[Transaction 3]
  Date:        03/10/2025
  Description: Point-of-Sale Transaction 93370950,SIMPLYGO PTE. LTD. NETS
  Withdrawal:  128.0
  Deposit:     -
  Balance:     219.5
  Page:        2

[Transaction 4]
  Date:        03/10/2025
  Description: Funds Transfer TOP-UP TO PAYLAH! : 90470220
  Withdrawal:  8.9
  Deposit:     -
  Balance:     210.6
  Page:        2

[Transaction 5]
  Date:        04/10/2025
  Description: FAST Payment / Receipt INCOMING PAYNOW REF 7663512 FROM: ONG...
  Withdrawal:  -
  Deposit:     20.26
  Balance:     230.86
  Page:        2


## 4. Convert to Pandas DataFrame

For easier analysis and manipulation, convert the transaction data to a pandas DataFrame.

In [5]:
# Create DataFrame
df = pd.DataFrame(result.data)

# Display basic info
print(f"Total Transactions: {len(df)}")
print(f"\nDataFrame Info:")
print(df.info())

# Display first few rows
print("\nFirst 10 Transactions:")
df.head(10)

Total Transactions: 36

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         36 non-null     object 
 1   description  36 non-null     object 
 2   withdrawal   24 non-null     float64
 3   deposit      12 non-null     float64
 4   balance      36 non-null     float64
 5   page         36 non-null     int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 1.8+ KB
None

First 10 Transactions:


,date,description,withdrawal,deposit,balance,page
0,01/10/2025,"FAST Payment / Receipt 1,900.00 UOB:7803927190...",NaN,NaN,337.70,2
1,01/10/2025,FAST Payment / Receipt INCOMING PAYNOW REF 305...,NaN,9.80,347.50,2
2,03/10/2025,"Point-of-Sale Transaction 93370950,SIMPLYGO PT...",128.0,NaN,219.50,2
3,03/10/2025,Funds Transfer TOP-UP TO PAYLAH! : 90470220,8.9,NaN,210.60,2
4,04/10/2025,FAST Payment / Receipt INCOMING PAYNOW REF 766...,NaN,20.26,230.86,2
5,04/10/2025,Funds Transfer TOP-UP TO PAYLAH! : 52880148A 9...,1.5,NaN,229.36,2
6,04/10/2025,FAST Payment / Receipt No. INCOMING PAYNOW REF...,NaN,18.80,248.16,2
7,06/10/2025,FAST Payment / Receipt 196800306E OCBC:7177025...,70.0,NaN,202.16,2
8,06/10/2025,FAST Payment / Receipt 196800306E OCBC:7177025...,70.0,NaN,202.16,2
9,06/10/2025,FAST Payment / Receipt INCOMING PAYNOW REF 126...,NaN,13.00,215.16,3


## 5. Data Analysis

Perform basic analysis on the extracted transactions.

In [6]:
# Summary statistics
print("Transaction Statistics:")
print("=" * 60)
print(f"Total Transactions:     {len(df)}")
print(f"Transactions with Withdrawals: {df['withdrawal'].notna().sum()}")
print(f"Transactions with Deposits:    {df['deposit'].notna().sum()}")
print(f"\nTotal Withdrawals:  SGD {df['withdrawal'].sum():,.2f}")
print(f"Total Deposits:     SGD {df['deposit'].sum():,.2f}")
print(f"Net Change:         SGD {df['deposit'].sum() - df['withdrawal'].sum():,.2f}")

# Opening and closing balances
opening_balance = df.iloc[0]['balance'] + (df.iloc[0]['withdrawal'] or 0) - (df.iloc[0]['deposit'] or 0)
closing_balance = df.iloc[-1]['balance']

print(f"\nOpening Balance:    SGD {opening_balance:,.2f}")
print(f"Closing Balance:    SGD {closing_balance:,.2f}")
print(f"Actual Change:      SGD {closing_balance - opening_balance:,.2f}")

Transaction Statistics:
Total Transactions:     36
Transactions with Withdrawals: 24
Transactions with Deposits:    12

Total Withdrawals:  SGD 3,848.29
Total Deposits:     SGD 380.08
Net Change:         SGD -3,468.21

Opening Balance:    SGD nan
Closing Balance:    SGD 101.06
Actual Change:      SGD nan


## 6. Filter and Search Transactions

Examples of how to filter and search through transactions.

In [7]:
# Find all deposits
deposits = df[df['deposit'].notna()]
print(f"Deposit Transactions ({len(deposits)}):")
print("=" * 100)
deposits[['date', 'description', 'deposit', 'balance']]

Deposit Transactions (12):


,date,description,deposit,balance
1,01/10/2025,FAST Payment / Receipt INCOMING PAYNOW REF 305...,9.80,347.50
4,04/10/2025,FAST Payment / Receipt INCOMING PAYNOW REF 766...,20.26,230.86
6,04/10/2025,FAST Payment / Receipt No. INCOMING PAYNOW REF...,18.80,248.16
9,06/10/2025,FAST Payment / Receipt INCOMING PAYNOW REF 126...,13.00,215.16
10,08/10/2025,FAST Payment / Receipt INCOMING PAYNOW REF 189...,15.20,230.36
12,10/10/2025,FAST Payment / Receipt INCOMING PAYNOW REF 642...,204.00,428.81
13,12/10/2025,FAST Payment / Receipt INCOMING PAYNOW REF 759...,41.00,469.81
14,12/10/2025,FAST Payment / Receipt INCOMING PAYNOW REF 886...,22.00,491.81
29,23/10/2025,Funds Transfer No: TOP-UP TO PAYLAH! : Reg MAR...,18.00,98.05
30,23/10/2025,Funds Transfer No: TOP-UP TO PAYLAH! : Reg MAR...,18.00,98.05


In [8]:
# Find large withdrawals (> 100 SGD)
large_withdrawals = df[df['withdrawal'] > 100]
print(f"Large Withdrawals > SGD 100 ({len(large_withdrawals)}):")
print("=" * 100)
large_withdrawals[['date', 'description', 'withdrawal', 'balance']]

Large Withdrawals > SGD 100 (3):


,date,description,withdrawal,balance
2,03/10/2025,"Point-of-Sale Transaction 93370950,SIMPLYGO PT...",128.00,219.50
15,13/10/2025,Payments / Collections via GIRO 52880148A SAF ...,3217.39,3709.20
25,21/10/2025,Payments / Collections via GIRO PRUDENTIAL PAC...,124.78,192.25


In [9]:
# Search for specific keywords in description
keyword = "PAYNOW"  # Change this to search for different keywords
matching_txns = df[df['description'].str.contains(keyword, case=False, na=False)]

print(f"Transactions containing '{keyword}' ({len(matching_txns)}):")
print("=" * 100)
matching_txns[['date', 'description', 'withdrawal', 'deposit', 'balance']]

Transactions containing 'PAYNOW' (12):


,date,description,withdrawal,deposit,balance
1,01/10/2025,FAST Payment / Receipt INCOMING PAYNOW REF 305...,NaN,9.80,347.50
4,04/10/2025,FAST Payment / Receipt INCOMING PAYNOW REF 766...,NaN,20.26,230.86
6,04/10/2025,FAST Payment / Receipt No. INCOMING PAYNOW REF...,NaN,18.80,248.16
9,06/10/2025,FAST Payment / Receipt INCOMING PAYNOW REF 126...,NaN,13.00,215.16
10,08/10/2025,FAST Payment / Receipt INCOMING PAYNOW REF 189...,NaN,15.20,230.36
12,10/10/2025,FAST Payment / Receipt INCOMING PAYNOW REF 642...,NaN,204.00,428.81
13,12/10/2025,FAST Payment / Receipt INCOMING PAYNOW REF 759...,NaN,41.00,469.81
14,12/10/2025,FAST Payment / Receipt INCOMING PAYNOW REF 886...,NaN,22.00,491.81
27,22/10/2025,FAST Payment / Receipt 52880148A PAYNOW TRANSF...,3.3,NaN,183.05
29,23/10/2025,Funds Transfer No: TOP-UP TO PAYLAH! : Reg MAR...,5.0,18.00,98.05


## 7. Validate Balance Continuity

Check that each transaction's balance follows correctly from the previous balance.

In [10]:
# Validate balance arithmetic
discrepancies = []

for i in range(1, len(df)):
    prev_balance = df.iloc[i-1]['balance']
    curr_balance = df.iloc[i]['balance']
    withdrawal = df.iloc[i]['withdrawal'] or 0
    deposit = df.iloc[i]['deposit'] or 0
    
    expected_balance = prev_balance - withdrawal + deposit
    
    # Allow for small floating point differences
    if abs(expected_balance - curr_balance) > 0.01:
        discrepancies.append({
            'index': i,
            'date': df.iloc[i]['date'],
            'expected': expected_balance,
            'actual': curr_balance,
            'difference': curr_balance - expected_balance
        })

if discrepancies:
    print(f"Found {len(discrepancies)} balance discrepancies:")
    for disc in discrepancies:
        print(f"  Transaction {disc['index']} ({disc['date']}): Expected {disc['expected']:.2f}, Got {disc['actual']:.2f}")
else:
    print("✓ All balance calculations are consistent!")

Found 2 balance discrepancies:
  Transaction 29 (23/10/2025): Expected 116.05, Got 98.05
  Transaction 30 (23/10/2025): Expected 111.05, Got 98.05


## 8. Export to JSON

Save the extracted data to a JSON file for further processing.

In [11]:
# Create output dictionary
output_data = {
    "success": result.success,
    "confidence": result.confidence,
    "warnings": result.warnings,
    "transaction_count": len(result.data),
    "data": result.data
}

# Save to file
output_path = "../extracted_data_notebook.json"
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"Data exported to: {output_path}")

Data exported to: ../extracted_data_notebook.json


## 9. Experiment: Parse Specific Pages

You can modify the parser to test specific scenarios or pages.

In [12]:
# Example: Get transactions from page 2 only
page_2_txns = df[df['page'] == 2]

print(f"Transactions on Page 2: {len(page_2_txns)}")
print("=" * 100)
page_2_txns

Transactions on Page 2: 9


,date,description,withdrawal,deposit,balance,page
0,01/10/2025,"FAST Payment / Receipt 1,900.00 UOB:7803927190...",NaN,NaN,337.70,2
1,01/10/2025,FAST Payment / Receipt INCOMING PAYNOW REF 305...,NaN,9.80,347.50,2
2,03/10/2025,"Point-of-Sale Transaction 93370950,SIMPLYGO PT...",128.0,NaN,219.50,2
3,03/10/2025,Funds Transfer TOP-UP TO PAYLAH! : 90470220,8.9,NaN,210.60,2
4,04/10/2025,FAST Payment / Receipt INCOMING PAYNOW REF 766...,NaN,20.26,230.86,2
5,04/10/2025,Funds Transfer TOP-UP TO PAYLAH! : 52880148A 9...,1.5,NaN,229.36,2
6,04/10/2025,FAST Payment / Receipt No. INCOMING PAYNOW REF...,NaN,18.80,248.16,2
7,06/10/2025,FAST Payment / Receipt 196800306E OCBC:7177025...,70.0,NaN,202.16,2
8,06/10/2025,FAST Payment / Receipt 196800306E OCBC:7177025...,70.0,NaN,202.16,2


## 10. Custom Analysis Playground

Use this cell to experiment with your own queries and analysis.

In [13]:
# Your custom analysis here

# Example: Group by date and sum amounts
# df['date_only'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
# daily_summary = df.groupby('date_only').agg({
#     'withdrawal': 'sum',
#     'deposit': 'sum'
# })
# print(daily_summary)